In [ ]:
!pip install stop-words
!pip install pymorphy2

In [24]:
import re
import numpy as np
import pandas as pd

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from string import punctuation

from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

# Задача Sentiment Analysis

Собраны твиты 2-ух тональностей, необходимо произвести классификацию на 2-а класса.

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import pandas as pd

#df_train = pd.read_csv("data/train.csv")
#df_test = pd.read_csv("data/test.csv")
#df_val = pd.read_csv("data/val.csv")

In [5]:
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/val.csv")


In [6]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [7]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [8]:
df_test.head()

,id,text
0,204150,Тектоника и рельеф-самое ужасное в мире мучение(
1,204151,"Ходили запускать шар желаний, но у нас не полу..."
2,204152,"Хочу лето только ради того, что бы направить н..."
3,204153,RT @RonyLiss: @colf_ne блин((\nа я шипперила Ф...
4,204154,"RT @anna_romt: @ZADROT_PO_IGRAM блин,каждое во..."


In [9]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


### Предобработка

In [12]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [13]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [14]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [15]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [16]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [18]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [19]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [20]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [21]:
x_train.shape

(181467, 40)

In [22]:
max_len

40

In [23]:
x_train[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1, 170,
         9], dtype=int32)

## Text classification using CNN

In [25]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [26]:
GlobalMaxPool1D?

In [27]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [29]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 48s 147ms/step - loss: 0.6295 - accuracy: 0.6155 - val_loss: 0.6169 - val_accuracy: 0.6250
Epoch 2/20
319/319 [==============================] - 50s 157ms/step - loss: 0.6159 - accuracy: 0.6269 - val_loss: 0.6159 - val_accuracy: 0.6297
Epoch 3/20
319/319 [==============================] - 45s 142ms/step - loss: 0.6124 - accuracy: 0.6303 - val_loss: 0.6150 - val_accuracy: 0.6251
Epoch 4/20
319/319 [==============================] - 45s 142ms/step - loss: 0.6090 - accuracy: 0.6349 - val_loss: 0.6153 - val_accuracy: 0.6251


In [30]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 36ms/step - loss: 0.6187 - accuracy: 0.6246


Test score: 0.6186949014663696
Test accuracy: 0.6245646476745605


In [31]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 2s 35ms/step


## Text classification using RNN

In [64]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [66]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [67]:
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [68]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [69]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [70]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 55s 165ms/step - loss: 0.5614 - accuracy: 0.6982 - val_loss: 0.4927 - val_accuracy: 0.7564
Epoch 2/10
319/319 [==============================] - 47s 148ms/step - loss: 0.2879 - accuracy: 0.8823 - val_loss: 0.5654 - val_accuracy: 0.7425


In [71]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 29ms/step - loss: 0.5983 - accuracy: 0.7309


Test score: 0.5983277559280396
Test accuracy: 0.7308557033538818


In [72]:
# Оценка производительности модели RNN на валидационных данных
score_rnn = model.evaluate(X_valid, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('RNN Test score:', score_rnn[0])
print('RNN Test accuracy:', score_rnn[1])

# Оценка производительности модели CNN на валидационных данных
score_cnn = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('CNN Test score:', score_cnn[0])
print('CNN Test accuracy:', score_cnn[1])

45/45 [==============================] - 1s 26ms/step - loss: 0.5983 - accuracy: 0.7309


RNN Test score: 0.5983277559280396
RNN Test accuracy: 0.7308557033538818
45/45 [==============================] - 2s 41ms/step - loss: 0.9326 - accuracy: 0.5327


CNN Test score: 0.9326282739639282
CNN Test accuracy: 0.5327337384223938


RNN -> CNN

In [ ]:
# model.add(Conv1D(128, 5, activation='relu'))

In [74]:
from keras.layers import Conv1D, GlobalMaxPooling1D

# Создание RNN -> CNN модели
model_rnn_cnn = Sequential()
model_rnn_cnn.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model_rnn_cnn.add(Masking(mask_value=0.0))

model_rnn_cnn.add(SimpleRNN(64, return_sequences=True))

# Добавление сверточных слоев
model_rnn_cnn.add(Conv1D(128, 3, activation='relu'))
model_rnn_cnn.add(GlobalMaxPooling1D())

# Добавление полносвязных слоев и выходного слоя
model_rnn_cnn.add(Dense(64, activation='relu'))
model_rnn_cnn.add(Dropout(0.5))
model_rnn_cnn.add(Dense(1, activation='sigmoid'))

# Компиляция модели
model_rnn_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [75]:
# Обучение модели RNN
history_rnn = model.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])

# Оценка производительности модели RNN на валидационных данных
score_rnn = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('RNN Test score:', score_rnn[0])
print('RNN Test accuracy:', score_rnn[1])

# Восстановление исходного состояния данных
df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/test.csv")
df_val = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/val.csv")

# Обучение модели RNN -> CNN
history_rnn_cnn = model_rnn_cnn.fit(X_train, y_train, batch_size=512, epochs=10, verbose=1, validation_split=0.1, callbacks=[early_stopping])

# Оценка производительности модели RNN -> CNN на валидационных данных
score_rnn_cnn = model_rnn_cnn.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('RNN -> CNN Test score:', score_rnn_cnn[0])
print('RNN -> CNN Test accuracy:', score_rnn_cnn[1])


Epoch 1/10
319/319 [==============================] - 49s 153ms/step - loss: 0.0878 - accuracy: 0.9688 - val_loss: 0.7976 - val_accuracy: 0.7271
Epoch 2/10
45/45 [==============================] - 1s 17ms/step - loss: 1.0211 - accuracy: 0.7171


RNN Test score: 1.0211411714553833
RNN Test accuracy: 0.7171450257301331
Epoch 1/10
319/319 [==============================] - 69s 207ms/step - loss: 0.5653 - accuracy: 0.6948 - val_loss: 0.4912 - val_accuracy: 0.7543
Epoch 2/10
45/45 [==============================] - 2s 46ms/step - loss: 0.5733 - accuracy: 0.7355


RNN -> CNN Test score: 0.5733462572097778
RNN -> CNN Test accuracy: 0.7354847192764282
